In [1]:
import pandas as pd
import re
import emoji
from ftfy import fix_text 
import unidecode
from langdetect import detect, detect_langs
import spacy

In [2]:
# df = pd.read_json(r'C:\Users\rockm\golden-globe-337\gg2013.json')['text']
df = pd.read_json('gg2013.json')['text']

In [26]:
df.to_csv('text.csv')

In [3]:
df

0                    JLo's dress! #eredcarpet #GoldenGlobes
1         What's making Sofia Vergara's boobs stay like ...
2         RT @FabSugar: Kerry Washington is EVERYTHING. ...
3            Anne Hathaway has got me living. #GoldenGlobes
4         Jennifer Lopez's lace dress? Thoughts? #Golden...
                                ...                        
174638    RT @authorViviAnna: I was sad that Mandy Patin...
174639    RT @_ItzelMartinez_: Jennifer Lawrence aceptan...
174640    Golden Globes, lots of fashion messes...but gl...
174641    Did they have mug shots at the golden globes?!...
174642    Says @BenAffleck: "I also didn't get the Actin...
Name: text, Length: 174643, dtype: object

In [28]:
def clean(text):
    text = fix_text(text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|pic.twitter\S+', '', text)
    # Remove emojis
    # new_text = unidecode(text)
    text = emoji.replace_emoji(text, replace='')
    # Remove hashtags
    text = re.sub(r'#\S+', '', text)
    
    text = re.sub(r'[^A-Za-z0-9@ "]+', '', text)
    
    text = re.sub(' +', ' ', text)
    
    text = text.lower()
    
    # if text.str.strip() != '':
    #     detect(text.split(' ')[0])
    
    return text

In [29]:
df = df.apply(clean)

In [30]:
cleaned_data = df[df.str.strip() != ""]

In [31]:
cleaned_data.to_csv('text_cleaned.csv')

In [11]:
cleaned_data = pd.read_csv('text_cleaned.csv')['text']

In [14]:
cleaned_data[cleaned_data.apply(lambda x: re.search('(?=.*award|AWARD)(?=.*wins|Wins|WINS|winner|WINNER).*', x) != None)]

1462      rt @possessionista if jennifer lawrence starts...
2944      id watch the but scripted tv wins out over awa...
6441      golden globes 2013 list of winners from this y...
6932                      bill murray wins the beard award 
8152      the begins hope django unchained wins an award...
                                ...                        
173191    rt @instyle looking every bit a winner stopped...
173233    golden globes 2013 british stars take home awa...
173291    golden globes 2013 british stars take home awa...
173330    rt @thr complete list of the winners at the 70...
173387    winner at our @goldenglobes party practice rou...
Name: text, Length: 646, dtype: object

In [68]:
win_data = cleaned_data[cleaned_data.apply(lambda x: re.search('(wins|Wins|WINS|winner|WINNER)', x) != None)]

In [75]:
win_data

125       "@gustavocadile oscar winner actress anjelica ...
349       too busy for the this go around somebody tweet...
549       jessica chastain looks like the mozilla firefo...
607       lmao "@morocca hey nbc shouldve gotten a "mani...
619       rt @morocca hey nbc shouldve gotten a "mani ca...
                                ...                        
173394    les miserables best picture musical or comedy ...
173408    listen to winners from the golden globes adele...
173439    jessica chastain best actress drama winner at ...
173460    all of the tv award winners at the are on cabl...
173466    rt @piersmorgan great amusingly hosted memorab...
Name: text, Length: 7495, dtype: object

In [16]:
from spacy import displacy

In [77]:
test_tweet = "rt @galaxiemag lifetime achievement award winner jodie foster is 50 she has been acting for 47 of those years amazing"

In [85]:
alleged_winner = ""
subject = []
actual_winner = ""

nlp = spacy.load('en_core_web_sm')

doc = nlp(test_tweet)

def extract_entities_2(doc):
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

for ent in extract_entities_2(doc):
    if ent[1] == "PERSON":
        alleged_winner = ent[0]

# for token in doc:
#     if (token.dep_ == 'nsubj'):
#         nsubj = token.text
#     elif (token.dep_ == 'dobj'):
#         dobj = token.text

for token in doc:
    # extract subject
    if (token.dep_ == 'nsubj'):
        subject.append(token.text)

In [95]:
def pattern(word, text): 
    pat = re.compile(re.escape(word), re.IGNORECASE) 
    return bool(re.search(pat, text))

In [98]:
entities = extract_entities_2(doc)
for s in subject:
    for e, _ in entities:
        if pattern(s, e):
            print(e)

jodie foster


In [81]:
doc.ents

(jodie foster, 50, 47)

In [102]:
nlp = spacy.load('en_core_web_sm')
def get_winner(text):
    alleged_winner = ""
    subject = []
    actual_winner = ""
    
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # for ent in entities:
    #     if ent[1] == "PERSON":
    #         alleged_winner = ent[0]
            
    for token in doc:
        # extract subject
        if (token.dep_ == 'nsubj'):
            subject.append(token.text)
    
    entities = extract_entities_2(doc)
    for s in subject:
        for e, _ in entities:
            if pattern(s, e):
                return e

In [103]:
len(win_data)

7495

In [119]:
win_data[607]

'lmao "@morocca hey nbc shouldve gotten a "mani cam" e wins this round '

In [111]:
win_data.head(250).apply(get_winner)

125       anjelica huston
349                  None
549      jessica chastain
607                   nbc
619           rt @morocca
               ...       
18022                None
18072                None
18088        maggie smith
18248        maggie smith
18287                None
Name: text, Length: 250, dtype: object

In [136]:
nom_data = cleaned_data[cleaned_data.apply(lambda x: re.search('(nominee|nominate|nominated|nominees)', x) != None)]

In [137]:
nom_data

49        rt @lokipage times we wish good luck tonight t...
61        rt @twittermovies nominee @officialadele on th...
154       ive seen a grand total of one nominated film s...
338       rt @zap2itrick can kerry washington be nominat...
505       rt @twittermovies nominee @officialadele on th...
                                ...                        
173059    the were amazing im so happy argo won but i th...
173107    rt @oskrnyc "7 cajas" should have been nominat...
173130    "@itsjerrah i should be nominated as best acto...
173172    congrats to all the 2013 golden globes winners...
173203    rt @mapeel yes should have been nominated for ...
Name: text, Length: 1385, dtype: object